In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PSD/TORONTO FILE

Mounted at /content/drive
/content/drive/MyDrive/PSD/TORONTO FILE


In [4]:
%%capture
pip install librosa

In [12]:
# Import Library

import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode

# **PEMROSESAN DATA**

## Deskripsi Data

Dataset ini untuk melatih klasifikasi emosi (7 emosi utama) dalam audio. Ada satu set 200 kata target yang diucapkan dalam frasa pembawa "Say the word _' oleh dua aktris (berusia 26 dan 64 tahun) dan rekaman dibuat dari set tersebut yang menggambarkan masing-masing tujuh emosi (marah, jijik, takut, bahagia, kejutan menyenangkan, kesedihan, dan netral). Total ada 2800 data (file audio).

Dataset tersebut disusun sedemikian rupa sehingga masing-masing dari dua aktor wanita dan emosi mereka terkandung dalam foldernya sendiri. Dan di dalamnya, semua file audio 200 kata target dapat ditemukan. Format file audio adalah format WAV

In [6]:
# Daftar folder yang akan dianalisis
folders = [
    'OAF_angry', 'OAF_disgust', 'OAF_Fear', 'OAF_happy', 'OAF_neutral', 'OAF_Pleasant_surprise', 'OAF_Sad', 'YAF_angry', 'YAF_disgust', 'YAF_fear', 'YAF_happy', 'YAF_neutral', 'YAF_pleasant_surprised', 'YAF_sad'
]

### **Menghitung Beberapa Statistik dari Sinyal Audio yang diberikan**

**Rumus-rumus dari hasil Ekstraksi Fitur Audio**

1.  Mean (Rata-rata)

  Rata-rata adalah nilai tengah dari suatu set data. Ini dihitung dengan menjumlahkan semua nilai dalam dataset dan kemudian membaginya dengan jumlah total data.

  $\text{{Mean}} = \frac{1}{n} \sum_{i=1}^{n} y_i$

2.  Standard Deviation (Deviasi Standar)

  Deviasi standar mengukur seberapa tersebar atau jauh nilai-nilai individu dalam dataset dari nilai rata-ratanya.

  $\text{{Standard Deviation}} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2}$

3. Median

  Median adalah nilai tengah dari suatu set data yang telah diurutkan. Jika jumlah elemen dalam dataset ganjil, median adalah nilai di tengah. Jika genap, median adalah rata-rata dari dua nilai tengah.

  $\text{{Median}} = \begin{cases}
      y_{(\frac{n}{2})} & \text{{if }} n \text{{ is odd}} \\
      \frac{1}{2} (y_{(\frac{n}{2})} + y_{(\frac{n}{2} + 1)}) & \text{{if }} n \text{{ is even}}
   \end{cases}$

4. Skewness

  Skewness mengukur sejauh mana distribusi data miring dari simetri. Nilai skewness positif menunjukkan ekor distribusi yang lebih panjang di sebelah kanan, sedangkan nilai negatif menunjukkan ekor distribusi yang lebih panjang di sebelah kiri.

  $\text{{Skewness}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^3}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^{3/2}}$

5. Kurtosis

  Kurtosis mengukur tingkat kecuraman (tumpukan atau landasan) distribusi data. Kurtosis tinggi menunjukkan distribusi yang lebih fokus di sekitar rata-rata, sementara kurtosis rendah menunjukkan distribusi yang lebih datar. Kurtosis dibandingkan dengan kurtosis distribusi normal standar yang memiliki nilai 3.

  $\text{{Kurtosis}} = \frac{\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^4}{\left(\frac{1}{n} \sum_{i=1}^{n} (y_i - \text{{Mean}})^2\right)^2} - 3$


1. Zero Crossing Rate (ZCR):

  Definisi: Nilai ini mengukur seberapa sering sinyal melintasi nilai nol per detik.

  **Rumus:**
  
  * zcr_mean: Nilai rata-rata ZCR.
  * zcr_median: Nilai median ZCR.
  * zcr_std_dev: Deviasi standar dari ZCR.
  * zcr_kurtosis: Kurtosis dari distribusi ZCR.
  * zcr_skew: Skewness dari distribusi ZCR.
  
2. Root Mean Square Error (RMSE):

  Definisi: RMSE mengukur rata-rata perbedaan antara nilai aktual dan nilai yang diprediksi (dalam hal ini, amplitudo sinyal audio).

  **Rumus:**

  * rmse: Nilai RMSE dari sinyal.
  * rmse_median: Nilai median dari RMSE.
  * rmse_std_dev: Deviasi standar dari RMSE.
  * rmse_kurtosis: Kurtosis dari distribusi RMSE.
  * rmse_skew: Skewness dari distribusi RMSE.

In [8]:
def calculate_statistics(audio_path):
    y, sr = librosa.load(audio_path)

    # UNTUK MENGHITUNG NILAI ZCR
    zcr_mean = np.mean(librosa.feature.zero_crossing_rate(y=y))
    zcr_median = np.median(librosa.feature.zero_crossing_rate(y=y))
    zcr_std_dev = np.std(librosa.feature.zero_crossing_rate(y=y))
    zcr_kurtosis = kurtosis(librosa.feature.zero_crossing_rate(y=y)[0])
    zcr_skew = skew(librosa.feature.zero_crossing_rate(y=y)[0])

    # UNTUK MENGHITUNG NILAI RMSE
    rmse = np.sum(y**2) / len(y)
    rmse_median = np.median(y**2)
    rmse_std_dev = np.std(y**2)
    rmse_kurtosis = kurtosis(y**2)
    rmse_skew = skew(y**2)

    return [zcr_mean, zcr_median, zcr_std_dev, zcr_kurtosis, zcr_skew, rmse, rmse_median, rmse_std_dev, rmse_kurtosis, rmse_skew]

In [9]:
# Membuat DataFrame untuk menyimpan hasil
data = []

In [10]:
# Loop melalui setiap folder dan file audio
for folder in folders:
    folder_path = f'{folder}'
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            statistics = calculate_statistics(audio_path)
            data.append([folder, filename] + statistics)

In [14]:
# Membuat DataFrame dari data
columns =  ['Label', 'File'] + ['ZCR Mean', 'ZCR Median', 'ZCR Std Dev', 'ZCR Kurtosis', 'ZCR Skew', 'RMSE', 'RMSE Median', 'RMSE Std Dev', 'RMSE Kurtosis', 'RMSE Skew']
df = pd.DataFrame(data, columns=columns)

In [15]:
# Menyimpan DataFrame ke dalam file CSV
df.to_csv('statistik_audio.csv', index=False)

In [16]:
# Menampilkan file CSV
df

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,OAF_angry,OAF_haze_angry.wav,0.088925,0.057861,0.076363,2.578202,1.795895,0.001001,0.000080,0.002729,55.999886,6.088586
1,OAF_angry,OAF_chat_angry.wav,0.106949,0.078613,0.094167,2.414703,1.814177,0.000954,0.000125,0.002312,34.864765,5.029483
2,OAF_angry,OAF_chain_angry.wav,0.109226,0.052734,0.117271,2.233586,1.804295,0.003233,0.000192,0.007497,16.470234,3.649717
3,OAF_angry,OAF_goose_angry.wav,0.118253,0.063721,0.115183,0.928318,1.495333,0.001733,0.000119,0.004727,31.237054,5.047650
4,OAF_angry,OAF_laud_angry.wav,0.087633,0.057617,0.098755,6.797438,2.719129,0.001401,0.000236,0.002905,30.049867,4.333659
...,...,...,...,...,...,...,...,...,...,...,...,...
2814,YAF_sad,YAF_voice_sad.wav,0.197648,0.054688,0.261351,-0.085103,1.283249,0.001513,0.000191,0.003156,17.681585,3.721640
2815,YAF_sad,YAF_pearl_sad.wav,0.112969,0.026855,0.211074,4.514809,2.463884,0.001323,0.000309,0.002921,34.923053,5.113679
2816,YAF_sad,YAF_shack_sad.wav,0.157730,0.039062,0.215866,1.586849,1.679327,0.001849,0.000383,0.003741,17.705001,3.776787
2817,YAF_sad,YAF_shawl_sad.wav,0.154858,0.038330,0.220376,1.509823,1.694042,0.001454,0.000397,0.002913,20.201689,3.995825


## **Penjelasan Label**

* 'OAF_angry': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi marah dari pembicara OAF.
*'OAF_disgust': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi jijik atau muak dari pembicara OAF.
*'OAF_Fear': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi takut dari pembicara OAF.
*'OAF_happy': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi bahagia dari pembicara OAF.
*'OAF_neutral': mencerminkan ekspresi wajah atau suara yang terkait dengan ekspresi wajah atau suara netral dari pembicara OAF.
*'OAF_Pleasant_surprise': mencerminkan ekspresi wajah atau suara yang terkait dengan kejutan menyenangkan dari pembicara OAF.
*'OAF_Sad': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi sedih dari pembicara OAF.
Untuk Pembicara YAF:
*'YAF_angry': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi marah dari pembicara YAF.
*'YAF_disgust': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi jijik atau muak dari pembicara YAF.
*'YAF_fear': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi takut dari pembicara YAF.
*'YAF_happy': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi bahagia dari pembicara YAF.
*'YAF_neutral': mencerminkan ekspresi wajah atau suara yang terkait dengan ekspresi wajah atau suara netral dari pembicara YAF.
*'YAF_Pleasant_surprise': mencerminkan ekspresi wajah atau suara yang terkait dengan kejutan menyenangkan dari pembicara YAF.
*'YAF_sad': mencerminkan ekspresi wajah atau suara yang terkait dengan emosi sedih dari pembicara YAF.

# **PREPROCESSING**

## **Normalisasi menggunakan Z-Score**

**Z-score** (Z-skor) adalah metode normalisasi yang mengubah data ke dalam distribusi normal standar dengan rata-rata (mean) 0 dan deviasi standar (standard deviation) 1.

**StandardScaler** adalah alat yang digunakan untuk menerapkan metode Z-score ini ke data Anda. Dengan kata lain, StandardScaler adalah implementasi konkret dari normalisasi Z-score dalam scikit-learn.

Rumus:

$z = \frac{{x - \mu}}{{\sigma}}$


Penjelasan:

*   Z adalah Z-score
*   x adalah nilai individu
*   μ adalah rata-rata dari data set
*   σ adalah deviasi standar dari data set

## **NORMALISASI SETELAH SPLIT DATA (normalisasi bisa dilakukan sebelum atau sesudah split data)**


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from pickle import dump

In [18]:
# Baca data dari file CSV
dataknn= pd.read_csv('statistik_audio.csv')
# Pisahkan fitur (X) dan label (y)
X = dataknn.drop(['Label','File'], axis=1)  # Ganti 'target_column' dengan nama kolom target
y = dataknn['Label']
# split data into train and test sets
X_train,X_test,y_train, y_test= train_test_split(X, y, random_state=1, test_size=0.2)
# define scaler
scaler = StandardScaler()
# fit scaler on the training dataset
scaler.fit(X_train)
# save the scaler
dump(scaler, open('scaler.pkl', 'wb'))
# transform the training dataset
# normalisasi data train
X_train_scaled = scaler.transform(X_train)
dataknn

,Label,File,ZCR Mean,ZCR Median,ZCR Std Dev,ZCR Kurtosis,ZCR Skew,RMSE,RMSE Median,RMSE Std Dev,RMSE Kurtosis,RMSE Skew
0,OAF_angry,OAF_haze_angry.wav,0.088925,0.057861,0.076363,2.578202,1.795895,0.001001,0.000080,0.002729,55.999886,6.088586
1,OAF_angry,OAF_chat_angry.wav,0.106949,0.078613,0.094167,2.414703,1.814177,0.000954,0.000125,0.002312,34.864765,5.029483
2,OAF_angry,OAF_chain_angry.wav,0.109226,0.052734,0.117271,2.233586,1.804295,0.003233,0.000192,0.007497,16.470234,3.649717
3,OAF_angry,OAF_goose_angry.wav,0.118253,0.063721,0.115183,0.928318,1.495333,0.001733,0.000119,0.004727,31.237054,5.047650
4,OAF_angry,OAF_laud_angry.wav,0.087633,0.057617,0.098755,6.797438,2.719129,0.001401,0.000236,0.002905,30.049867,4.333659
...,...,...,...,...,...,...,...,...,...,...,...,...
2814,YAF_sad,YAF_voice_sad.wav,0.197648,0.054688,0.261351,-0.085103,1.283249,0.001513,0.000191,0.003156,17.681585,3.721640
2815,YAF_sad,YAF_pearl_sad.wav,0.112969,0.026855,0.211074,4.514809,2.463884,0.001323,0.000309,0.002921,34.923053,5.113679
2816,YAF_sad,YAF_shack_sad.wav,0.157730,0.039062,0.215866,1.586849,1.679327,0.001849,0.000383,0.003741,17.705001,3.776787
2817,YAF_sad,YAF_shawl_sad.wav,0.154858,0.038330,0.220376,1.509823,1.694042,0.001454,0.000397,0.002913,20.201689,3.995825


In [19]:
X_test.shape

(564, 10)

In [20]:
X_train.shape

(2255, 10)

In [21]:
import pickle
with open('scaler.pkl', 'rb') as standarisasi:
    loadscal= pickle.load(standarisasi)

In [22]:
#normalisasi X testing dari hasil normalisasi X train yang disimpan dalam model
X_test_scaled=loadscal.transform(X_test)

# **METODE KNN**

**K-Nearest Neighbors (KNN)** adalah algoritma pembelajaran mesin yang digunakan untuk tugas klasifikasi dan regresi. Algoritma ini merupakan salah satu metode pembelajaran mesin yang termasuk dalam kategori pembelajaran berbasis instan (instance-based learning) atau pembelajaran berbasis data.

Klasifikasi dengan KNN:
Dalam konteks klasifikasi, KNN bekerja dengan cara sebagai berikut:

* Penentuan Tetangga Terdekat: Mengukur jarak antara data baru (poin uji) dengan semua titik data latih.
* Pemilihan K Tetangga Terdekat: Memilih K tetangga terdekat dari data baru berdasarkan jarak.
* Voting atau Weighted Voting: Menentukan label atau nilai target untuk data baru dengan melihat mayoritas kelas dari K tetangga terdekat. Dalam beberapa implementasi, bobot dapat diterapkan pada voting berdasarkan jarak.

**Rumus:**

$\hat{y} = \text{majority}(\{y_i \,|\, (x_i, y_i) \in \text{k-nearest neighbors of } x\})$


In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [24]:
K = 50
acc = np.zeros((K-1))

for n in range(1,K,2):
    knn = KNeighborsClassifier(n_neighbors= n, metric = "euclidean").fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)

    acc[n-1]= accuracy_score(y_test,y_pred)

print('akurasi terbaik adalah ', acc.max(), 'dengan nilai k =', acc.argmax()+1)

akurasi terbaik adalah  0.6985815602836879 dengan nilai k = 3


In [25]:
knn = KNeighborsClassifier(n_neighbors= 7, metric = "euclidean")
#dump(knn, open('modelknn.pkl', 'wb'))

In [27]:
#import pickle
#with open('modelknn.pkl', 'rb') as knn:
#    loadknn= pickle.load(knn)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [28]:
y_pred = knn.predict(X_test_scaled)
y_pred

array(['YAF_angry', 'YAF_pleasant_surprised', 'OAF_Pleasant_surprise',
       'YAF_neutral', 'YAF_happy', 'YAF_sad', 'YAF_neutral',
       'YAF_disgust', 'YAF_happy', 'OAF_Sad', 'OAF_angry', 'OAF_angry',
       'OAF_neutral', 'YAF_angry', 'YAF_pleasant_surprised',
       'YAF_pleasant_surprised', 'YAF_disgust', 'YAF_sad', 'OAF_Sad',
       'YAF_neutral', 'OAF_neutral', 'YAF_sad', 'OAF_happy',
       'YAF_disgust', 'OAF_disgust', 'YAF_happy', 'OAF_Pleasant_surprise',
       'YAF_pleasant_surprised', 'OAF_neutral', 'OAF_angry',
       'OAF_Pleasant_surprise', 'OAF_Fear', 'OAF_disgust', 'OAF_neutral',
       'YAF_neutral', 'YAF_fear', 'YAF_pleasant_surprised', 'OAF_Fear',
       'YAF_neutral', 'YAF_fear', 'OAF_Pleasant_surprise', 'YAF_disgust',
       'YAF_neutral', 'YAF_pleasant_surprised', 'YAF_disgust',
       'OAF_angry', 'OAF_disgust', 'YAF_neutral', 'YAF_happy', 'YAF_fear',
       'YAF_disgust', 'YAF_happy', 'YAF_sad', 'YAF_angry', 'YAF_fear',
       'OAF_Fear', 'YAF_disgust', 'OAF_

In [29]:
accuracy = accuracy_score(y_test,y_pred)
print("Akurasi:",accuracy)

Akurasi: 0.6932624113475178


In [30]:
dump(knn, open('modelknn.pkl', 'wb'))

# **REDUKSI DATA**

Reduksi ada 2 jenis bisa menggunakan seleksi data atau transformation data (contoh nya PCA / Principal component anlysis).
seleksi data kita dapat emmilih fitur berdasarkan yang fitur/ kolom yang paling berpengaruh.
sedangkan transformasi data kita perlu membuat kooordinat baru dari dari fitur yang ada sejumlah dengan jumlah fitur nya.
mengapa reduksi data atau data reduction di perlukan?
karna terlalu banyak kolom/fitur/ciri yang harus dikenali tidak baik untuk pemprosesan data dan memakan waktu komputasi yang lama, maka dari itu reduksi data di lakukan untuk mendapatkan data yang terbaik.
ketika ingin mencari koordinat baru:
1. buat matriks covarian
2. buat persamaan

note : konstanta yang paling besar mengartikan di koordinat tersebut merupakan ciri paling banyak atau penting

In [31]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=8)
X_train_pca= sklearn_pca.fit_transform(X_train_scaled)
type(X_train_pca)

numpy.ndarray

In [32]:
dump(sklearn_pca, open('PCA1.pkl', 'wb'))

In [34]:
import pickle
with open('PCA1.pkl', 'rb') as pca:
    loadpca= pickle.load(pca)

In [35]:
X_test_pca=loadpca.transform(X_test_scaled)
X_test_pca.shape

(564, 8)

In [36]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=15)
classifier.fit(X_train_pca, y_train)

KNeighborsClassifier(n_neighbors=15)

In [37]:
y_prediksi = classifier.predict(X_test_pca)
y_prediksi

array(['YAF_angry', 'YAF_fear', 'OAF_Pleasant_surprise', 'YAF_neutral',
       'YAF_happy', 'YAF_sad', 'YAF_neutral', 'YAF_pleasant_surprised',
       'YAF_happy', 'OAF_Sad', 'OAF_angry', 'OAF_angry', 'OAF_neutral',
       'YAF_angry', 'YAF_pleasant_surprised', 'YAF_pleasant_surprised',
       'YAF_disgust', 'YAF_sad', 'OAF_Sad', 'YAF_neutral', 'OAF_neutral',
       'YAF_sad', 'OAF_Fear', 'YAF_disgust', 'OAF_disgust', 'YAF_happy',
       'OAF_Pleasant_surprise', 'YAF_pleasant_surprised', 'OAF_neutral',
       'OAF_angry', 'OAF_Pleasant_surprise', 'OAF_Fear', 'OAF_disgust',
       'OAF_neutral', 'YAF_neutral', 'YAF_angry', 'YAF_disgust',
       'OAF_Fear', 'YAF_neutral', 'YAF_fear', 'OAF_Pleasant_surprise',
       'YAF_neutral', 'YAF_neutral', 'YAF_pleasant_surprised',
       'YAF_disgust', 'OAF_angry', 'OAF_disgust', 'YAF_neutral',
       'YAF_happy', 'YAF_fear', 'YAF_disgust', 'YAF_happy', 'YAF_sad',
       'YAF_angry', 'YAF_fear', 'OAF_Pleasant_surprise', 'YAF_disgust',
       'OAF_P

In [38]:
acc_pca= accuracy_score(y_test,y_prediksi)
print("Akurasi:",acc_pca)

Akurasi: 0.6861702127659575
